# Title: CORRELATION MODEL IN THE ADOPTION OF E-PAYMENT SERVICES

## Load Libraries

In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as ipw
import plotly.express as px
import scipy.stats as stats
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from sklearn import svm, tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, precision_recall_curve, roc_curve

from scipy.stats import pearsonr, spearmanr, kendalltau, pointbiserialr, chi2, chi2_contingency

from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier

from dtreeviz.trees import *

%matplotlib inline

## Load Custom Made Libraries

In [3]:
from Utilities.CFS import *
from Utilities.accuracy import *
from Utilities.corr_matrix import *
from Utilities.forward_selection import *
from Utilities.backward_elimination import *
from Utilities.prob_distributed_feature import *

from Visualization.model_graph import *
from Visualization.network_graph import *

## Load Dataset

In [7]:
df = pd.read_csv("test.csv")

In [16]:
df.columns.tolist()

['No.',
 'Timestamp',
 'I have read and agreed with the terms above.',
 'Year of birth ',
 'Gender ',
 'Ethnicity ',
 'Nationality',
 'Marital Status ',
 'Occupation ',
 'Monthly Income ',
 'Education level ',
 'Telemedicine improves my access to healthcare services.',
 'Telemedicine saves me time traveling to a hospital or specialist clinic. ',
 'Telemedicine provides for my healthcare needs. ',
 'It was simple to use this system. ',
 'It was easy to learn to use the system. ',
 'The way I interact with this system is pleasant. ',
 'I like using this system. ',
 'This system is simple and easy to understand. ',
 'I could easily talk to the clinician using the telehealth system. ',
 'I could hear the clinician clearly using the telehealth system.',
 'I felt I was able to express myself effectively. ',
 'Using the telemedicine system, I can see the clinician as well as if we met in person. ',
 'I think the visits provided over the telemedicinesystem are the same as in-person visits. ',


In [17]:
f_str = "None"
t_str = "Gender ->Ethnicity ->Nationality->Marital Status ->Occupation ->Monthly Income ->Education level "
u_str = "I like using this system. ->This system is simple and easy to understand. ->I could easily talk to the clinician using the telehealth system. ->I could hear the clinician clearly using the telehealth system.->I felt I was able to express myself effectively. ->Using the telemedicine system, I can see the clinician as well as if we met in person. ->I think the visits provided over the telemedicinesystem are the same as in-person visits. ->Whenever I made a mistake using the system, I could recover easily and quickly. ->The system gave error messages that clearly told me how to fix problems. ->Telehealth is an acceptable way to receive healthcare services.->I feel comfortable communicating with the clinician using the telemedicine system.->Telemedicine is an acceptable way to receive healthcare services.->Overall, I am satisfied with this telehealth system."

In [37]:
filter_feature = f_str.split("->")

if filter_feature == ["None"]:
    filter_feature = []

targeted_feature = t_str.split("->")
utaut_feature = u_str.split("->")

In [38]:
df.loc[:, filter_feature]

In [39]:
df.loc[:, targeted_feature]

,Gender,Ethnicity,Nationality,Marital Status,Occupation,Monthly Income,Education level
0,Female,Malay,Malaysian,Married,Professional,> RM 10000,Diploma
1,Male,Chinese,Non-Malaysian,Married,Retired,RM 5000 - RM 10000,Secondary school
2,Female,Kadazan,Malaysian,Single,Professional,Rm 1200- RM 5000,Bachelor's Degree
3,Female,Chinese,Malaysian,Married,Freelancer,Rm 1200- RM 5000,Bachelor's Degree
4,Female,Chinese,Malaysian,Married,Freelancer,Rm 1200- RM 5000,Bachelor's Degree
...,...,...,...,...,...,...,...
195,Female,Chinese,Malaysian,Married,Professional,Rm 1200- RM 5000,Bachelor's Degree
196,Female,Chinese,Malaysian,Divorced,Professional,> RM 10000,Master/PhD/Others
197,Male,Chinese,Malaysian,Married,Businessman,RM 5000 - RM 10000,Diploma
198,Female,Chinese,Malaysian,Single,Professional,Rm 1200- RM 5000,Bachelor's Degree


In [40]:
df.loc[:, utaut_feature]

,I like using this system.,This system is simple and easy to understand.,I could easily talk to the clinician using the telehealth system.,I could hear the clinician clearly using the telehealth system.,I felt I was able to express myself effectively.,"Using the telemedicine system, I can see the clinician as well as if we met in person.",I think the visits provided over the telemedicinesystem are the same as in-person visits.,"Whenever I made a mistake using the system, I could recover easily and quickly.",The system gave error messages that clearly told me how to fix problems.,Telehealth is an acceptable way to receive healthcare services.,I feel comfortable communicating with the clinician using the telemedicine system.,Telemedicine is an acceptable way to receive healthcare services.,"Overall, I am satisfied with this telehealth system."
0,5,5,5,5,5,5,5,5,5,5,5,5,5
1,5,5,5,5,5,5,4,5,5,4,5,5,5
2,5,5,5,5,5,4,5,5,5,5,5,4,5
3,5,5,5,5,5,3,4,5,5,5,5,4,5
4,5,5,5,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,1,1,2,1,2,2,2,1,1,1,1,1
196,5,5,5,5,5,5,4,5,5,5,5,5,5
197,5,5,5,5,5,5,5,5,5,5,5,5,5
198,3,5,4,5,3,3,2,3,3,3,3,3,3


## Generate Unique For Each Feature Variable

In [41]:
def df_unique(arr):
    tmp_keys = []
    for val in arr:
        if type(val) != str:
            continue
        tmp_list = val.split(";")
        for val2 in tmp_list:
            if val2 not in tmp_keys:
                tmp_keys.append(val2)
                
    arr_list = []
    for val in arr:
        if type(val) != str:
            continue
        tmp_dict = {i:"No" for i in tmp_keys}
        tmp_list = val.split(";")
        for val2 in tmp_list:
            tmp_dict[val2] = "Yes"
        arr_list.append(tmp_dict)
    
    return pd.DataFrame(arr_list)

In [42]:
filter_feature_df = df.loc[:, filter_feature]

In [46]:
filter_feature_res_df = pd.DataFrame()
arr_df = []
feature_dict = {}
for feat in filter_feature_df.columns:
    tmp_df = df_unique(filter_feature_df.loc[:, feat])
    if len(tmp_df.columns.tolist()) == 2:
        filter_feature_res_df[feat] = filter_feature_df.loc[:, feat]
        feature_dict[feat] = ["None"]
    else:
        feature_dict[feat] = tmp_df.columns.tolist()
        tmp_df.columns = [f"{feat}->{val}" for val in tmp_df.columns]
        arr_df.append(tmp_df)
        display(tmp_df)
        
feature_dict["None"] = ["None"]

In [47]:
arr_df.append(filter_feature_res_df)

In [48]:
feature_dict

{'None': ['None']}

In [49]:
filter_feature_res_df = pd.concat(arr_df, axis = 1)

filter_feature_res_df

""


In [52]:
targeted_feature_df = df.loc[:, targeted_feature]
utaut_feature_df = df.loc[:, utaut_feature]

In [56]:
arr_df = [targeted_feature_df, filter_feature_res_df, utaut_feature_df]
df = pd.concat(arr_df, axis = 1)

df

,Gender,Ethnicity,Nationality,Marital Status,Occupation,Monthly Income,Education level,I like using this system.,This system is simple and easy to understand.,I could easily talk to the clinician using the telehealth system.,I could hear the clinician clearly using the telehealth system.,I felt I was able to express myself effectively.,"Using the telemedicine system, I can see the clinician as well as if we met in person.",I think the visits provided over the telemedicinesystem are the same as in-person visits.,"Whenever I made a mistake using the system, I could recover easily and quickly.",The system gave error messages that clearly told me how to fix problems.,Telehealth is an acceptable way to receive healthcare services.,I feel comfortable communicating with the clinician using the telemedicine system.,Telemedicine is an acceptable way to receive healthcare services.,"Overall, I am satisfied with this telehealth system."
0,Female,Malay,Malaysian,Married,Professional,> RM 10000,Diploma,5,5,5,5,5,5,5,5,5,5,5,5,5
1,Male,Chinese,Non-Malaysian,Married,Retired,RM 5000 - RM 10000,Secondary school,5,5,5,5,5,5,4,5,5,4,5,5,5
2,Female,Kadazan,Malaysian,Single,Professional,Rm 1200- RM 5000,Bachelor's Degree,5,5,5,5,5,4,5,5,5,5,5,4,5
3,Female,Chinese,Malaysian,Married,Freelancer,Rm 1200- RM 5000,Bachelor's Degree,5,5,5,5,5,3,4,5,5,5,5,4,5
4,Female,Chinese,Malaysian,Married,Freelancer,Rm 1200- RM 5000,Bachelor's Degree,5,5,5,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Female,Chinese,Malaysian,Married,Professional,Rm 1200- RM 5000,Bachelor's Degree,1,1,1,2,1,2,2,2,1,1,1,1,1
196,Female,Chinese,Malaysian,Divorced,Professional,> RM 10000,Master/PhD/Others,5,5,5,5,5,5,4,5,5,5,5,5,5
197,Male,Chinese,Malaysian,Married,Businessman,RM 5000 - RM 10000,Diploma,5,5,5,5,5,5,5,5,5,5,5,5,5
198,Female,Chinese,Malaysian,Single,Professional,Rm 1200- RM 5000,Bachelor's Degree,3,5,4,5,3,3,2,3,3,3,3,3,3


## Write JSON to File

In [ ]:
import json

with open("dict_to_json_textfile.txt", 'w') as fout:
    json_dumps_str = json.dumps(a_dictionary, indent=4)
    print(json_dumps_str, file=fout)